In [1]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
import pandas as pd
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard

In [2]:
df = pd.read_csv ('/Users/abby/Documents/fa21-neural-networks/final_project/all-the-news-2-1.csv')

/Users/abby/Applications/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'date', 'year', 'month', 'day', 'author',
       'title', 'article', 'url', 'section', 'publication'],
      dtype='object')

In [4]:
df[df['publication']=='Fox News'].groupby('year').count()

,Unnamed: 0,Unnamed: 0.1,date,month,day,author,title,article,url,section,publication
year,,,,,,,,,,,
2016,56,56,56,56,56,56,56,56,56,56,56
2017,2464,2464,2464,2464,2464,2464,2464,2464,2464,2464,2464
2018,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624


In [5]:
df_fox_news = df[df['publication'] == "Fox News"]
df_fox_news = df_fox_news[df_fox_news['year'] == 2018]
df_fox_news = df_fox_news[['article']]
df_fox_news.to_csv('fox_news.csv', index=False)

In [6]:
df[df['year'] == 2018].groupby('publication').count()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section
publication,,,,,,,,,,,
Axios,16543,16543,16543,16543,16543,16543,16542,16543,16512,16543,0
Business Insider,2204,2204,2204,2204,2204,2204,2165,2204,2202,2204,0
Buzzfeed News,8194,8194,8194,8194,8194,8194,8194,8194,8174,8194,0
CNBC,42965,42965,42965,42965,42965,42965,404,42964,40685,42965,30927
CNN,29736,29736,29736,29736,29736,29736,28821,29736,29262,29736,29736
Economist,5942,5942,5942,5942,5942,5942,0,5942,5635,5942,5942
Fox News,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624
Gizmodo,6590,6590,6590,6590,6590,6590,6590,6590,6590,6590,5593
Hyperallergic,3045,3045,3045,3045,3045,3045,3044,3045,3036,3045,0


In [7]:
df_cnn = df[df['publication'] == "CNN"]
df_cnn = df_cnn[df_cnn['year'] == 2018]
df_cnn = df_cnn[['article']]
df_cnn.to_csv('cnn.csv', index=False)

In [8]:
df_the_hill = df[df['publication'] == "The Hill"]
df_the_hill = df_the_hill[df_the_hill['year'] == 2018]
df_the_hill = df_the_hill[['article']]
df_the_hill.to_csv('the_hill.csv', index=False)

In [10]:
df_reuters = df[df['publication'] == "Reuters"]
df_reuters = df_reuters[df_reuters['year'] == 2018]
df_reuters = df_reuters[['article']]
df_reuters.to_csv('reuters.csv', index=False)

In [13]:
df_nyt = df[df['publication'] == "The New York Times"]
df_nyt = df_nyt[df_nyt['year'] == 2018]
df_nyt = df_nyt[['article']]
df_nyt.to_csv('nyt.csv', index=False)